In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install pysastrawi
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 4.2 MB/s eta 0:00:00
/kaggle/input/tokopedia-product-and-review-dataset/tokopedia_products_with_review.json
/kaggle/input/tokopedia-product-and-review-dataset/tokopedia_products_with_review.csv
/kaggle/input/stopwords/stopwords.txt


In [2]:
# df = pd.read_csv('/kaggle/input/tokopedia-product-and-review-dataset/tokopedia_products_with_review.csv')
df = pd.read_json('/kaggle/input/tokopedia-product-and-review-dataset/tokopedia_products_with_review.json')
dataset_list = df.head(50).to_dict(orient='records')

In [3]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

filename = '/kaggle/input/stopwords/stopwords.txt'

stopwords = []

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Open the file in read mode ('r') using a context manager
with open(filename, 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Optional: Use rstrip() to remove trailing whitespace/newlines
        processed_line = line.strip()
        text = stemmer.stem(processed_line)
        
        # Process the line (e.g., print it)
        if(processed_line and text not in stopwords):
            stopwords.append(text)

len(dataset_list)

50

In [4]:
type(dataset_list)
reviews = list(map(lambda review: review['reviews'], dataset_list))
reviews = [item for sublist in reviews for item in sublist]
reviews = list(map(lambda review: {
    'message': review['message'],
    'rating': review['review_rating'],
    'sentiment': 'positive' if review['review_rating'] > 3 else 'negative'
}, reviews))
df = pd.DataFrame(reviews)
print(df.shape)
# for review in reviews:
#     print(review['rating'], review['sentiment'], review['message'])
# df

(9271, 3)


In [5]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re



def remove_stopword(sentence): 
    arr = sentence.split()
    resArr = []
    for text in arr:
        if(text not in stopwords):
            resArr.append(text)
    return ' '.join(resArr)


def handle_negation_enhanced(text):
    """Enhanced negation handling"""
    
    # More comprehensive negation list
    negations = [
        'tidak', 'tak', 'bukan', 'jangan', 'belum', 'tanpa',
        'gak', 'ga', 'engga', 'nggak', 'kagak', 'ngg', 'ngga',  # ← Slang variations!
        'gk', 'g', 'tdk'  # Abbreviations
    ]
    
    for neg in negations:
        # More flexible pattern (handle spacing issues)
        pattern = r'\b' + neg + r'\s+(\w+)'
        text = re.sub(pattern, r'not \1', text)
    
    return text

def preprocess_text(text):
    """
    Preprocessing steps:
    1. Lowercase
    2. Remove special characters
    3. Remove stopwords
    4. Stemming
    """
    # Lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    text = stemmer.stem(text)

    return handle_negation_enhanced(text)


clean_message = []
for review in reviews:
    # clean_message.append(remove_stopword(preprocess_text(review['message'])))
    clean_message.append((preprocess_text(review['message'])))
df['clean_message'] = pd.Series(clean_message)
df = df[['message', 'clean_message', 'rating', 'sentiment']]

indices_to_drop = df[df['sentiment'] == 'positive'].index[:7500]

# 3. Hapus baris berdasarkan indeks tersebut
df = df.drop(indices_to_drop)


df['sentiment'].value_counts()

sentiment
positive    1044
negative     727
Name: count, dtype: int64

In [6]:
# Features and target
X = df['clean_message']  # ← Use cleaned text!
y = df['sentiment']

class_counts = y.value_counts()
for sentiment, count in class_counts.items():
    percentage = (count / len(y)) * 100
    print(f"  {sentiment}: {count} ({percentage:.1f}%)")

if class_counts.min() / class_counts.max() < 0.5:
    print("\n⚠️  WARNING: Data is imbalanced!")
    print("  Consider using class_weight='balanced' in models")

  positive: 1044 (58.9%)
  negative: 727 (41.1%)


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_message'])
X



feature_names = vectorizer.get_feature_names_out()
df_tfidf = pd.DataFrame(
    X[:10].toarray(),  # First 5 docs
    columns=feature_names,
    index=[f"Doc_{i}" for i in range(10)]
)

df_tfidf.insert(0, 'clean_message', df['clean_message'][:10].values)
df_tfidf
# df_tfidf.to_csv('/tf-idf-word-2.csv', index=False)

,clean_message,aamiin,abang,abang kurir,abg,abg kurir,abis,abis sikat,abis siram,abs,...,yg udah,yg umumny,yg untuk,yg varian,yingkatkan,yingkatkan lagi,you,you can,you seller,zonk
Doc_0,ini dua kali nya akhir saya beli krn yg pertam...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_1,besi untuk pasang ke dinding not di amplas dan...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_2,sayang sekali ada pecah sekitar cm not tau dar...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_3,ada yg bentur cacat tapi gpp sdikit lah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_4,kecewa sih pertama karena lama bngt paket dua ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_5,buruk salah kirim warna jawab suka padahal di ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_6,barang cukup bagus hanya saja kerangka ambal k...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_7,terima barang ada penyok dikit di ujung lain k...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_8,warna not sesuai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc_9,lumayan bagus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# train model
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 31163 stored elements and shape (1771, 13107)>

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

def process_and_convert_text(text):
    text = (preprocess_text(text))

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2,random_state=42,stratify=y)


models = {
    'svc' : SVC(kernel='linear', probability=True, random_state=42),
    'naive_bayes': MultinomialNB(),
    'logistic_regression': LogisticRegression(max_iter=1000, random_state=42),
    'random_forest': RandomForestClassifier(n_estimators=100, random_state=42),
}

results = {}
train_indices = y_train.index
test_indices = y_test.index


for name, model in models.items():
    print(f"model {name}")

    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

     # TRAIN SET COMPARISON
    df_train_comparison = pd.DataFrame({
        'message': df['message'].loc[train_indices].values,
        'sentiment_original': y_train.values,
        'sentiment_pred': y_train_pred
    })
    train_accuracy = round(accuracy_score(y_train, y_train_pred) * 100, 2)
    test_accuracy = round(accuracy_score(y_test, y_test_pred) * 100, 2)
    print(f"train_accuracy is {train_accuracy}\ntest_accuracy is {test_accuracy}")
    print("="*70)

text_test = [
    {
        'message': 'Bangsat jancuk, ga guna barangnya',
        'expect': 'negative'
    },
    {
        'message': 'Sangat berguna',
        'expect': 'positive'
    },
    {
        'message': 'Barang tidak berguna sama sekali',
        'expect': 'negative'
    },
    {
        'message': 'Terlalu mahal kualitas jelek',
        'expect': 'negative',
    },
    {
        'message': 'produk bagus',
        'expect': 'positive'
    }, 
    {
        'message': 'produk tidak bagus',
        'expect': 'negative'
    },
    {
        'message': 'sangat bagus',
        'expect': 'positive'
    },
    {
        'message': 'tidak bagus sama sekali',
        'expect': 'negative'
    },
    {
        'message': 'produk tolol, seller tidak ramah',
        'expect': 'negative'
    },
    {
        'message': 'aku suka belanja disini, barang lengkap dan seller responsif',
        'expect': 'positive'
    },
    {
        'message': 'aku suka produk ini, sangat membantu dan berguna untuk sehari-hari',
        'expect': 'positive'
    },
    {
        'message': 'hari yg indah, membuat hati ingin liburan dan membeli produk yang sangat bagus ini, terimakasih tokped',
        'expect': 'positive'
    }
]



num_correct_model = {
    'svc': 0,
    'naive_bayes': 0,
    'logistic_regression': 0,
    'random_forest': 0
}

for test in text_test:
    v = vectorizer.transform([(preprocess_text(test['message']))]) 
    expect = test['expect']
    svc_predict = models['svc'].predict(v)[0]
    naive_bayes_predict = models['naive_bayes'].predict(v)[0]
    logistic_regression_predict = models['logistic_regression'].predict(v)[0]
    random_forest_predict = models['random_forest'].predict(v)[0]

    svc_correct = expect == svc_predict
    naive_bayes_correct = expect == naive_bayes_predict
    logistic_regression_correct = expect == logistic_regression_predict
    random_forest_correct = expect == random_forest_predict

    if(svc_correct): 
        num_correct_model['svc'] = num_correct_model['svc'] + 1 
    if(naive_bayes_correct):
        num_correct_model['naive_bayes'] = num_correct_model['naive_bayes'] + 1  
    if(logistic_regression_correct):
        num_correct_model['logistic_regression'] = num_correct_model['logistic_regression'] + 1 
    if(random_forest_correct):
        num_correct_model['random_forest'] = num_correct_model['random_forest'] + 1  
    
    print('SVC: ', test['message'], '=>' , svc_predict,  '✅' if svc_correct else '❌')
    print('Naive Bayes: ',test['message'], '=>', naive_bayes_predict, '✅' if naive_bayes_correct else '❌')
    print('Logistic Regression: ',test['message'], '=>', logistic_regression_predict, '✅' if logistic_regression_correct else '❌')
    print('Random Forest: ', test['message'], '=>', random_forest_predict, '✅' if random_forest_correct else '❌')
    print('='*70)

for model, num_correct in num_correct_model.items():
    print(f"accuracy for {model} is {round(num_correct / len(text_test) * 100, 2)}")

model svc
train_accuracy is 96.61
test_accuracy is 86.2
model naive_bayes
train_accuracy is 91.1
test_accuracy is 85.35
model logistic_regression
train_accuracy is 91.53
test_accuracy is 86.48
model random_forest
train_accuracy is 97.1
test_accuracy is 86.2
SVC:  Bangsat jancuk, ga guna barangnya => negative ✅
Naive Bayes:  Bangsat jancuk, ga guna barangnya => positive ❌
Logistic Regression:  Bangsat jancuk, ga guna barangnya => negative ✅
Random Forest:  Bangsat jancuk, ga guna barangnya => negative ✅
SVC:  Sangat berguna => positive ✅
Naive Bayes:  Sangat berguna => positive ✅
Logistic Regression:  Sangat berguna => positive ✅
Random Forest:  Sangat berguna => positive ✅
SVC:  Barang tidak berguna sama sekali => negative ✅
Naive Bayes:  Barang tidak berguna sama sekali => negative ✅
Logistic Regression:  Barang tidak berguna sama sekali => negative ✅
Random Forest:  Barang tidak berguna sama sekali => negative ✅
SVC:  Terlalu mahal kualitas jelek => negative ✅
Naive Bayes:  Terlalu m

In [10]:
class_pct = df['sentiment'].value_counts(normalize=True) * 100
class_pct.max() / class_pct.min()
preprocess_text('bangsat jancuk ga guna barang')

'bangsat jancuk not guna barang'

In [11]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score


print("="*70)
print("CROSS-VALIDATION (5-FOLD)")
print("="*70)


# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, pos_label='positive', average='binary'),
    'recall': make_scorer(recall_score, pos_label='positive', average='binary'),
    'f1': make_scorer(f1_score, pos_label='positive', average='binary')
}

models = {
    'svc' : SVC(kernel='linear', probability=True, random_state=42),
    'naive_bayes': MultinomialNB(),
    'logistic_regression': LogisticRegression(max_iter=1000, random_state=42),
    'random_forest': RandomForestClassifier(n_estimators=100, random_state=42),
}


dict_accuracy_model = {}
for name, model in models.items():
    print(f"model {name}")

     # Perform cross-validation
    cv_scores = cross_validate(
        model, 
        X,  # Full dataset
        y,
        cv=5,  # 5-fold
        scoring=scoring,
        return_train_score=True,
        n_jobs=-1  # Use all CPU cores
    )

    # Calculate statistics
    for metric in ['accuracy']:
        train_key = f'train_{metric}'
        test_key = f'test_{metric}'
        
        train_mean = cv_scores[train_key].mean()
        train_std = cv_scores[train_key].std()
        test_mean = cv_scores[test_key].mean()
        test_std = cv_scores[test_key].std()

        rounded_test_mean = round(test_mean * 100, 2)
        rounded_test_std = round(test_std * 100, 2)
        dict_accuracy_model[name] = (rounded_test_mean, rounded_test_std)
        
        print(f"metric {metric}")
        print(f"train_mean : {round(train_mean * 100, 2)}")
        print(f"train_std : {round(train_std * 100, 2)}")
        print(f"test_mean : {round(test_mean * 100 , 2)}")
        print(f"test_std : {round(test_std * 100, 2)}")
        
    print('='*70)
    
max_model = max(dict_accuracy_model.items(), key=lambda item: item[1][0])
best_model_name = max_model[0]
print(f"best model is {best_model_name} with {max_model[1][0]} +- {max_model[1][1]}")


CROSS-VALIDATION (5-FOLD)
model svc
metric accuracy
train_mean : 96.68
train_std : 0.25
test_mean : 85.32
test_std : 2.09
model naive_bayes
metric accuracy
train_mean : 91.38
train_std : 0.65
test_mean : 79.33
test_std : 2.05
model logistic_regression
metric accuracy
train_mean : 91.73
train_std : 0.63
test_mean : 81.88
test_std : 3.35
model random_forest
metric accuracy
train_mean : 97.11
train_std : 0.36
test_mean : 82.89
test_std : 2.1
best model is svc with 85.32 +- 2.09


In [12]:

models_for_testing = {
    'svc' : SVC(kernel='linear', probability=True, random_state=42),
    'naive_bayes': MultinomialNB(),
    'logistic_regression': LogisticRegression(max_iter=1000, random_state=42),
    'random_forest': RandomForestClassifier(n_estimators=100, random_state=42),
}



model_final = models_for_testing[best_model_name]
model_final.fit(X, y)
y_pred = model_final.predict(X)

count_correct = 0
for index, x in enumerate(y_pred):
    if x == list(y)[index]:
        count_correct+=1
    
print(f"train accuracy is {count_correct/len(y) * 100}")

print('='*70)

num_correct_test = 0
for test in text_test:
    v = vectorizer.transform([(preprocess_text(test['message']))]) 
    expect = test['expect']
    final_model_predict = model_final.predict(v)[0]
    

    final_model_correct = expect == final_model_predict
    
    
    
    if(final_model_correct): 
        num_correct_test = num_correct_test + 1 
    
    
    print(f'model {best_model_name}', test['message'], '=>' , final_model_predict,  '✅' if final_model_correct else '❌')
    
    print('='*70)

print(f"accuracy for {best_model_name} using test case is {round(num_correct_test / len(text_test) * 100 ,2)}")

train accuracy is 96.61208356860531
model svc Bangsat jancuk, ga guna barangnya => negative ✅
model svc Sangat berguna => positive ✅
model svc Barang tidak berguna sama sekali => negative ✅
model svc Terlalu mahal kualitas jelek => negative ✅
model svc produk bagus => positive ✅
model svc produk tidak bagus => negative ✅
model svc sangat bagus => positive ✅
model svc tidak bagus sama sekali => negative ✅
model svc produk tolol, seller tidak ramah => negative ✅
model svc aku suka belanja disini, barang lengkap dan seller responsif => positive ✅
model svc aku suka produk ini, sangat membantu dan berguna untuk sehari-hari => positive ✅
model svc hari yg indah, membuat hati ingin liburan dan membeli produk yang sangat bagus ini, terimakasih tokped => positive ✅
accuracy for svc using test case is 100.0
